# Analysis: Synthetic Control Method (SCM)

This notebook loads the dataset, constructs a synthetic control for unit `u0`, shows diagnostics, and computes treatment effect estimates.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import nnls

# Load data
df = pd.read_csv('data/dataset.csv')
treated_unit = 'u0'
treatment_time = 60

# Prepare matrices
pre = df[df.time < treatment_time]
post = df[df.time >= treatment_time]

# Pivot outcomes: rows=time, cols=unit
Y = df.pivot(index='time', columns='unit', values='outcome').sort_index()
Y_pre = Y.loc[Y.index < treatment_time]

# Donor pool (exclude treated)
donors = [c for c in Y.columns if c != treated_unit]

# Solve non-negative least squares to find weights that best reconstruct treated pre-treatment
y_treated = Y_pre[treated_unit].values
X_donors = Y_pre[donors].values
w, rnorm = nnls(X_donors, y_treated)
w = w / w.sum()  # normalize to sum to 1
weights = pd.Series(w, index=donors)
print('Donor weights:\n', weights)

# Construct synthetic control
Y_synth = Y[donors].dot(weights)

# Plot actual vs synthetic
plt.figure(figsize=(10,5))
plt.plot(Y.index, Y[treated_unit], label='Treated (u0)')
plt.plot(Y.index, Y_synth, label='Synthetic')
plt.axvline(treatment_time-0.5, color='k', linestyle='--', label='Treatment')
plt.legend()
plt.title('Actual vs Synthetic Control for u0')
plt.xlabel('time')
plt.ylabel('outcome')
plt.show()

# Compute treatment effect series and average post-treatment effect
effect = Y[treated_unit] - Y_synth
avg_post_effect = effect.loc[effect.index >= treatment_time].mean()
print('Average post-treatment effect (mean):', float(avg_post_effect))


## Placebo test (brief)
Run the same SCM procedure for each donor as if it were treated at the same time, compute placebo effects, and compare magnitudes and RMSPE ratios.


## Notes
- The notebook uses `scipy.optimize.nnls` to enforce non-negative weights and then normalizes them. Alternative formulations (convex optimization with `cvxpy`) are also possible.
- For reproducible grading, include figures of pre-treatment fit, RMSPE numbers, and placebo distributions.
